<a href="https://colab.research.google.com/github/DmitryKutsev/DeepHW/blob/master/hw3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install youtokentome

     |████████████████████████████████| 1.7MB 12.2MB/s 


In [5]:
# !wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
# !dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
# !apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
# !apt-get update
# !apt-get install cuda

In [6]:
# !pip install mxnet-cu92

In [2]:
print(torch.cuda.device_count())

1


In [3]:
print(torch.cuda.get_device_name(0))

Tesla T4


In [9]:
device = torch.device('cuda:0')

In [ ]:
# !pip install --upgrade --force-reinstall -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.0-{platform}-linux_x86_64.whl torchvision

In [1]:
import math
import numpy as np
import pandas as pd

import random
import json
import torch
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm

from matplotlib import pyplot as plt

import youtokentome as yttm

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
!unzip '/content/drive/My Drive/unsupervised.csv.zip'

Archive:  /content/drive/My Drive/unsupervised.csv.zip
  inflating: unsupervised.csv        


In [20]:
questions = pd.read_csv('unsupervised.csv')
questions.question = questions.question.map(lambda x: x.lower())

In [21]:
# для "обучения" bpe модели нам нужно сохранить данные для обучения в отдельный файл
# где будут построчно храниться тексты
# раскомментируйте этот код, чтобы собрать такой файл
with open('for_bpe.txt', 'w', encoding='utf-8') as f:
    for que in questions.question:
        f.write(que + '\n')
        
# параметры
vocab_size = 30_000
model_path = 'pretrained_bpe_lm.model'

In [22]:
%%time
# обучаем
# раскомментируйте этот код, чтобы обучить bpe
yttm.BPE.train(data='for_bpe.txt', vocab_size=vocab_size, model=model_path)

CPU times: user 23.1 s, sys: 1.84 s, total: 25 s
Wall time: 15.2 s


In [23]:
# загружаем токенизатор
tokenizer = yttm.BPE(model=model_path)

In [24]:
tokenized = []

batch_size = 256

for i_batch in tqdm(range(math.ceil(len(questions.question) / batch_size))):
    
    tokenized.extend(tokenizer.encode(
        list(questions.question[i_batch*batch_size:(i_batch+1)*batch_size]), bos=True, eos=True))

100%|██████████| 9905/9905 [00:29<00:00, 341.25it/s]


In [25]:
class LanguageModelData(torch.utils.data.Dataset):
    
    def __init__(self, data, max_len, pad_index, eos_index):
        
        self.data = data
        
        self.max_len = max_len
        
        self.pad_index = pad_index
        self.eos_index = eos_index
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        
        sequence = self.data[index][:self.max_len]
        
        # исходная последовательность
        x = sequence[:]
        # нужно предсказать смещенную последовательность
        y = sequence[1:] + [self.eos_index]
        
        assert len(x) == len(y)
        
        pads = [self.pad_index] * (self.max_len - len(x))
        
        x = torch.tensor(x + pads).long()
        y = torch.tensor(y + pads).long()
        
        return x, y

In [26]:
batch_size = 64

max_len = 32

pad_index = 0
eos_index = 3

In [27]:
# надо выбрать максимальную длину
lengths = np.array([len(x) for x in tokenized])
np.percentile(lengths, q=95)

28.0

In [28]:
random.shuffle(tokenized)

validation_start_index = int(len(tokenized) * 0.05)

In [29]:
train_dataset = LanguageModelData(data=tokenized[:-validation_start_index], max_len=max_len, 
                                  pad_index=pad_index, eos_index=eos_index)
validation_dataset = LanguageModelData(data=tokenized[-validation_start_index:], max_len=max_len,
                                       pad_index=pad_index, eos_index=eos_index)

len(train_dataset), len(validation_dataset)

(2408671, 126772)

In [30]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)
validation_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=batch_size)

In [31]:

for x, y in train_loader:
    break

In [32]:
x

tensor([[    2,  1917,  1775,  ...,     0,     0,     0],
        [    2,  2254, 25690,  ...,     0,     0,     0],
        [    2,  2930, 11900,  ...,     0,     0,     0],
        ...,
        [    2,  1960, 13584,  ...,     0,     0,     0],
        [    2,  1864,  6695,  ...,     0,     0,     0],
        [    2,  1816,  2018,  ...,     0,     0,     0]])

In [33]:
progress_bar = tqdm(total=len(validation_loader.dataset), desc='Testing')

for x, y in validation_loader:
    progress_bar.update(x.size(0))
    
progress_bar.close()

Testing: 100%|██████████| 126772/126772 [00:01<00:00, 69456.03it/s]


In [34]:
class LanguageModel(torch.nn.Module):
    
    def __init__(self, vocab_size, embedding_dim, model_dim, num_layers, dropout, padding_idx):
        
        super().__init__()
        
        # просто эмбеддинги
        self.embedding_layer = torch.nn.Embedding(num_embeddings=vocab_size, 
                                                  embedding_dim=embedding_dim,
                                                  padding_idx=padding_idx)
        
        # просто lstm
        self.lstm = torch.nn.LSTM(input_size=embedding_dim, 
                                  hidden_size=model_dim,
                                  num_layers=num_layers, 
                                  dropout=0.3,
                                  batch_first=True)
        
        # выходная матрица эмбеддингов
        # количество выходных фичей равно размеру словаря
        # то есть это задача мультиклассовой классификации, но только классов очень много
        self.language_model_head = torch.nn.Linear(in_features=model_dim,
                                                   out_features=vocab_size,
                                                   bias=False)
        
    def forward(self, x):
        
        x = self.embedding_layer(x)
        
        x, _ = self.lstm(x)
        
        # к каждому элементу последовательности применяется выходная матрица эмбеддингов, 
        # которая переводит вектор lstm в предсказание конкретного слова
        x = self.language_model_head(x)
        
        return x

In [35]:
torch.cuda.is_available(), 'у вас не находится гпу'

(True, 'у вас не находится гпу')

In [39]:
device = torch.device('cuda:0')

In [40]:
embedding_dim = 128
model_dim = 128
num_layers = 2
dropout = 0.35

In [41]:
model = LanguageModel(vocab_size=vocab_size, embedding_dim=embedding_dim,
                      model_dim=model_dim, num_layers=num_layers,
                      dropout=dropout, padding_idx=pad_index)
model.to(device)

LanguageModel(
  (embedding_layer): Embedding(30000, 128, padding_idx=0)
  (lstm): LSTM(128, 128, num_layers=2, batch_first=True, dropout=0.3)
  (language_model_head): Linear(in_features=128, out_features=30000, bias=False)
)

In [42]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [43]:
print(f'Количество обучаемых параметров в сети: {count_parameters(model):,}')

Количество обучаемых параметров в сети: 7,944,192


In [44]:
x = x.to(device)
y = y.to(device)